### load required packages

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json

### load data

In [ ]:
#if need to load from file
X_df_all_new = pd.read_csv('./X_df_all.csv')
y_df_all= pd.read_csv('./y_df_all.csv')
X_df_new = X_df_all_new.drop(columns = ['Unnamed: 0'])
X = X_df_new.values
y_df_all1 = y_df_all.total_price
y_df_all1[y_df_all1.isna()==True]=0
y_df_all1[y_df_all1!= 0]= np.log(y_df_all1[y_df_all1 != 0])
y = y_df_all1.values

### Heckman correction

In [ ]:
import copy
select_data = copy.copy(X_df_new)
select_data['price']= y
select_data['transaction_date']=pd.to_datetime(u_assets['transaction_time']).dt.to_period('D')
select_data['transaction_week']=u_assets['week']
select_data1 = select_data[select_data['price']!=0]
select_y = select_data1['price'].values
select_data2 = select_data1.drop(columns = ['price','transaction_week','transaction_date'])
select_X = select_data2.values

# heckman correction (probit model)
from statsmodels.discrete.discrete_model import Probit
probit_y = select_data['price']!=0
# select X included in probit model
H_X = X_df_new.drop(columns = X_df_new.iloc[:,84:239])
X1 = H_X.iloc[:,:84].values
model = Probit(probit_y, X1)
probit_model = model.fit()
print(probit_model.summary())

### comparison

In [ ]:
#OLS model on truncated data
import statsmodels.api as sm
mod = sm.OLS(select_y, select_X)
reg0 = mod.fit()
coeff_a = reg0.params
print(reg0.summary())

In [ ]:
#corrected model
mod = sm.OLS(select_y, correct_X)
reg = mod.fit()
h_coef = reg.params
heckman_coef = pd.DataFrame(h_coef)
print(reg.summary())

### Huberization

In [ ]:
heckman_est_y = np.dot(correct_X,heckman_coef.iloc[:,0])
heckman_res = y[y!=0]- heckman_est_y

#### loss function

In [ ]:
def psi_fun(x, kappa_upper, kappa_lower):
    if abs(x)<= kappa_upper and abs(x) >= kappa_lower:
        y= x
    elif x< kappa_lower:
        y= kappa_lower
    else:
        y= kappa_upper
    return(y)

#### define threshold  

In [ ]:
kappa_09_heckman = np.quantile(heckman_res,0.9)
kappa_01_heckman = np.quantile(heckman_res,0.1)
kappa_095_heckman = np.quantile(heckman_res,0.95)
kappa_005_heckman = np.quantile(heckman_res,0.05)
kappa_099_heckman = np.quantile(heckman_res,0.99)
kappa_001_heckman = np.quantile(heckman_res,0.01)
kappa_09_ols = np.quantile(reg0.resid,0.9)
kappa_01_ols = np.quantile(reg0.resid,0.1)
kappa_095_ols = np.quantile(reg0.resid,0.95)
kappa_005_ols = np.quantile(reg0.resid,0.05)
kappa_099_ols = np.quantile(reg0.resid,0.99)
kappa_001_ols = np.quantile(reg0.resid,0.01)

#### Huberizing

In [ ]:
deno_heckman = np.array([item >kappa_005_heckman and item <kappa_095_heckman for item in heckman_res]).sum()
deno_OLS = np.array([item >kappa_005_ols and item <kappa_095_ols for item in reg0.resid]).sum()
huber_trans_Heckmanres= [psi_fun(item,kappa_upper=kappa_095_heckman, kappa_lower=kappa_005_heckman) for item in heckman_res]
huber_trans_OLSres= [psi_fun(item,kappa_upper=kappa_095_ols, kappa_lower=kappa_005_ols) for item in reg0.resid]
huber_Heckmanres = np.array(huber_trans_Heckmanres)*len(heckman_res)/deno_heckman
huber_OLSres = np.array(huber_trans_OLSres)*len(reg0.resid)/deno_OLS
huber_yHeckman = heckman_est_y +huber_trans_Heckmanres
huber_yOLS = reg0.predict(select_X) +huber_OLSres